# Detección de Anomalías en Logs HDFS con PyTorch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/alex-msu/anomaly-log-detector/blob/main/notebooks/anomaly_detection.ipynb)

Este notebook ejecuta todo el pipeline de detección de anomalías en logs HDFS usando un autoencoder en PyTorch.

## Configuración inicial

In [ ]:
# Conectar con Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Clonar el repositorio del proyecto
repo_url = "https://github.com/alex-msu/anomaly-log-detector.git"
!git clone {repo_url}
%cd anomaly-log-detector

In [ ]:
# Instalar dependencias
!pip install -r requirements.txt

In [ ]:
# Verificar el entorno
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## Ejecutar el pipeline completo

In [ ]:
# Ejecutar el proceso principal
!python main.py

## Visualización de resultados

In [ ]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
# Cargar resultados
mse_scores = joblib.load('data/results/mse_scores.joblib')
preds = joblib.load('data/results/anomaly_preds.joblib')
meta = joblib.load('data/results/anomaly_meta.joblib')

In [ ]:
# Mostrar métricas
print("\nMétricas de detección:")
print(f"- Threshold: {meta['threshold']:.4f}")
print(f"- Método: {meta['threshold_method']}")
print(f"- Porcentaje de anomalías: {meta['anomaly_ratio']*100:.2f}%")
print(f"- Total anomalías detectadas: {int(meta['num_anomalies'])}")

In [ ]:
# Histograma de MSE
plt.figure(figsize=(10, 6))
plt.hist(mse_scores, bins=50, alpha=0.7, color='blue')
plt.axvline(meta['threshold'], color='red', linestyle='dashed', linewidth=2)
plt.title('Distribución de Errores de Reconstrucción (MSE)')
plt.xlabel('MSE')
plt.ylabel('Frecuencia')
plt.legend(['Umbral de anomalía', 'Distribución MSE'])
plt.grid(True)
plt.show()


In [ ]:
# Matriz de confusión y reporte de clasificación (si tenemos etiquetas reales)
try:
    y_true = joblib.load('data/processed/y.joblib')

    # Matriz de confusión
    cm = confusion_matrix(y_true, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normal', 'Anomalía'])
    disp.plot(cmap='Blues', values_format='d')
    plt.title('Matriz de Confusión')
    plt.show()

    # Reporte de clasificación
    print("\nReporte de Clasificación:")
    print(classification_report(y_true, preds, target_names=['Normal', 'Anomalía']))

    # Curva Precision-Recall
    from sklearn.metrics import precision_recall_curve, auc
    precision, recall, _ = precision_recall_curve(y_true, mse_scores)
    pr_auc = auc(recall, precision)

    plt.figure(figsize=(10, 6))
    plt.plot(recall, precision, color='darkorange', lw=2, label=f'PR curve (AUC = {pr_auc:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Curva Precision-Recall')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

except Exception as e:
    print(f"No se pudieron cargar las etiquetas reales para evaluación completa: {e}")

## Guardar resultados en Drive

In [ ]:
# Crear directorio en Drive
drive_base = '/content/drive/MyDrive/anomaly_detection_results'
!mkdir -p {drive_base}

# Copiar resultados
!cp -r data/results {drive_base}
!cp -r models {drive_base}
!cp main.py {drive_base}
!cp requirements.txt {drive_base}

print(f"Resultados guardados en: {drive_base}")

## Exportar notebook como script

In [ ]:
!jupyter nbconvert --to script anomaly_detection.ipynb

print("\n¡Proceso completo! Puedes encontrar todos los resultados en Google Drive.")